In [10]:
import pandas as pd
import numpy as np
import os
import translate

# Dataset 1a: German → English Argumentation Reviews (Thiemo)

1. Train a German GloVE model on these embeddings from column 'review'
2. WEAT Analysis
3. Word Co-occurence analysis

In [5]:
!pip install nltk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 10.5 MB/s eta 0:00:00m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.8/281.8 KB 12.5 MB/s eta 0:00:00


In [11]:
import nltk
from nltk.corpus import stopwords

german_stop_words = stopwords.words('german')

In [12]:
data_path = 'background/'
student_reviews_data = pd.read_excel(data_path + '201901_Studenten Reviews BI2 full.xlsx', header=1, index_col=0)
student_reviews_data

,itpa (Learning Unit),year,authorID (Ersteller des Reviews),reviewerID (Wer wurde gereviewt),reviewNumFromChange,review,Q_completed_task,Q_constructive_suggestions,Q_critical_aspects,Q_easy_understand,Q_high_quality,Q_highlights_weaknesses_strengths,Q_task_related,Q_was_helpful,review_list
0,1,2016,dominik.geissler@student.unisg.ch,boletin.asani@unisg.ch,1,Geschäftsidee HC19 Die Idee eine Art MetaPlat...,2,6,6,4,6,4,4,6,geschäftsidee hc19 die idee eine art metaplatt...
1,1,2016,dominik.geissler@student.unisg.ch,arne.meeuw@unisg.ch,3,Feedback zur Beschreibung / AllgemeinKonzept g...,2,7,6,4,6,4,4,6,feedback zur beschreibung allgemeinkonzept gef...
2,1,2016,dominik.geissler@student.unisg.ch,patrik.brueggemann@student.unisg.ch,2,1 Eine zentrale App für das Buchen von Tickets...,2,5,5,4,5,4,4,5,1 eine zentrale app für da buchen von tickets ...
3,2,2016,marc.burgunder@student.unisg.ch,clarajohanna.schruff@student.unisg.ch,2,"Die Ausführungen sind sehr gut, detailliert un...",2,3,3,4,2,4,4,3,die ausführungen sind sehr gut detailliert und...
4,2,2016,marc.burgunder@student.unisg.ch,fabian.rados@student.unisg.ch,1,Worin sehen Sie die Stärken der Lösungdes Komm...,2,5,4,4,5,4,4,5,worin sehen sie die stärken der lösungdes komm...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5157,6,2017,felix.meindl@student.unisg.ch,delia.wyss@student.unisg.ch,3,Worin sehen Sie die Stärken der Lösungdes Komm...,2,5,6,6,5,5,6,6,worin sehen sie die stärken der lösungdes komm...
5158,6,2017,felix.meindl@student.unisg.ch,gabriel.barmettler@student.unisg.ch,1,"Könnte Teils etwas ausführlicher sein, enthält...",2,4,6,5,5,3,5,6,könnte teils etwas ausführlicher sein enthält ...
5159,6,2017,terence.la@student.unisg.ch,bastian.zarskebueno@student.unisg.ch,3,Liebe/r Kommilitone/inEs war sehr interessant ...,2,4,6,5,5,5,5,4,liebe/r kommilitone/ines war sehr interessant ...
5160,6,2017,terence.la@student.unisg.ch,marco.sansoni@student.unisg.ch,2,Du hast mit deinem Geschäftsmodellbeschrieb ei...,2,3,5,5,5,5,3,4,du hast mit deinem geschäftsmodellbeschrieb ei...


In [32]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [38]:
import re
import string

def stop_word_removal(x):
    token = x.split()
    cleaned = ' '.join([w for w in token if not w in german_stop_words])
    return cleaned.replace('.', '\n')

def clean_text(text):
    # remove numbers
    text_nonum = re.sub(r'\d+', '', text)
    # remove punctuations and convert characters to lower case
    text_nopunct = "".join([char.lower() for char in text_nonum]) 
    # substitute multiple whitespace with single whitespace
    # Also, removes leading and trailing whitespaces
    text_no_doublespace = re.sub('\s+', ' ', text_nopunct).strip()
    return text_no_doublespace

student_reviews_data['review'] = student_reviews_data['review'].apply(stop_word_removal)
student_reviews_data['review'] = student_reviews_data['review'].apply(clean_text)
full_review = '\n'.join(student_reviews_data['review'])
with open("reviews_glove.txt", "w") as text_file:
    text_file.write(full_review)

# Dataset 1b: German Argumentation Reviews (Thiemo)

0. Translate text to English
1. Train an English GloVE model on these embeddings from column 'review'
2. WEAT Analysis
3. Word Co-occurence analysis

benefits: direct comparison to baselines from the original paper  
cons: how much bias can be attributed to the content, and how much to the translator?

In [ ]:
import deep_translator
from deep_translator import GoogleTranslator

translator = GoogleTranslator(source='de', target='en')

def translate_5000(text):
    characters = len(text)
    print(characters > 3000)
    translations = []
    for i in np.arange(3000, characters, 3000):
        translation = translator.translate(text[i-3000:i])
        translations.append(translation)
    translations.append(translator.translate(text[:-characters%3000]))
    return ''.join(translations)

student_reviews_data['review'] = student_reviews_data['review'].apply(lambda k: translate_5000(k))

In [ ]:
student_reviews_data.to_csv('english_translated_student_reviews.csv')

# Dataset 2: German Annotated Essays

1. Train an German GloVE model on these embeddings from student essays
2. WEAT Analysis
3. Word Co-occurence analysis
4. Analyze using annotations

In [39]:
data_path = 'background/Coprus Coling/Annotations/'
coling_essay_files = [file for file in os.listdir(data_path) if ".txt" in file]
essays = {}
for essay_file in coling_essay_files:
    with open(data_path+essay_file, "r") as myfile:
        data = myfile.read().splitlines()
        data_cleaned = clean_text(stop_word_removal(data[1]))
        essays[int(essay_file.split('.txt')[0])] = data[1]

In [40]:
all_essays = '\n'.join(essays.values())

with open("essays_glove.txt", "w") as text_file:
    text_file.write(all_essays)

# Dataset 3: English Annotated Essays

1. Train an English GloVE model on these embeddings from student essays
2. WEAT Analysis
3. Word Co-occurence analysis
4. Analyze using annotations 

In [41]:
data_path = 'background/ArgumentAnnotatedEssays-2.0/brat-project-final/'
eng_essay_files = [file for file in os.listdir(data_path) if ".txt" in file]
essays = {}
for essay_file in eng_essay_files:
    with open(data_path+essay_file, "r") as myfile:
        data = myfile.read()
        data_cleaned = stop_word_removal(clean_text(data))
        essays[int(essay_file.split('.txt')[0].split('essay')[1])] = data_cleaned
essays

{22: "young people should go to university or not persistent deliberation is happening as to whether it is desirable for the youth to go to university\n regard to the development of the country, some people believe that it is essential for young students to get high quality of education universities, while others think that it is only one of causes to increase the rate of unemployment\n this essay, i analyze both views first order to come up with a fool-proof decision\n to begin with, those who advocate the impracticability of university have several reasons\n from their point of view, the fact is that there are numerous job opportunities for those who do not have the certification of university graduation, such as mechanics, sales persons, or engineers, and, furthermore, labors for graduates are short supply\n what is worse, the labor market for highly educated students is now too competitive to get a job after finishing their studying\n this regard it is a loss of national productivi

In [42]:
all_essays = '\n'.join(essays.values())
with open("eng_essays_glove.txt", "w") as text_file:
    text_file.write(all_essays)

In [43]:
len(essays.values())

402

# WEAT Co-occurence Analysis

In [2]:
import spacy

In [ ]:
!python -m spacy download en_core_web_sm
!python -m spacy download de_core_news_sm

In [44]:
!python weat_cooccurrence_analysis.py \
    --data "eng_essays_glove.txt" \
    --output "output/weat_cooccurrence_analysis" \
    --processing_cores 7 \
    --tests 1 2 3 4 5 6 7 8 9 10

INFO:2022-04-05 23:14:35:Please make sure that your input texts are whitespace separated tokens. The script might not work correctly otherwise.
INFO:2022-04-05 23:14:35:Generating target-association test pairs...
INFO:2022-04-05 23:14:35:Extracting candidate posts...
100%|████████████████████████████████████| 12538/12538 [00:17<00:00, 733.15it/s]
INFO:2022-04-05 23:14:53:Splitting candidate posts into sentences...
100%|███████████████████████████████████████████| 7/7 [00:00<00:00, 8814.21it/s]
INFO:2022-04-05 23:14:59:Calculating sentence-based cooccurrences...
12538it [00:01, 8993.25it/s]
INFO:2022-04-05 23:15:00:Unpacking cooccurrences into a more useful format...
INFO:2022-04-05 23:15:00:Exporting results to disk at output/weat_cooccurrence_analysis/weat-cooccurrence-analysis_results-20220405231500.json.
Done.


In [47]:
!python accumulate_cooccurrence_counts.py \
    --input "output/weat_cooccurrence_analysis/weat-cooccurrence-analysis_results-20220405231500.json" \
    --weat_lexicons "sbeval/tests/weat_tests.json"

INFO:2022-04-05 23:17:49:Printing accumulated results for corpus 'eng_essays_glove.txt'.


Current test: test1
---------------------------------------------
X Flowers
Association-A:Association-B
0:0
Y insects
Association-A:Association-B
0:0


Current test: test2
---------------------------------------------
X Instruments
Association-A:Association-B
0:0
Y weapons
Association-A:Association-B
0:0


Current test: test3
---------------------------------------------
X European-American
Association-A:Association-B
0:0
Y African-American names
Association-A:Association-B
0:0


Current test: test4
---------------------------------------------
X European-American
Association-A:Association-B
0:0
Y African-American names
Association-A:Association-B
0:0


Current test: test5
---------------------------------------------
X European-American
Association-A:Association-B
0:0
Y African-American names
Association-A:Association-B
0:0


Current test: test6
---------------------------------------------
X Ma

In [45]:
!python weat_cooccurrence_analysis_german.py \
    --data "reviews_glove.txt" \
    --output "output/weat_cooccurrence_analysis" \
    --processing_cores 7 \
    --tests 1 2 3 4 5 6 7 8 9 10

INFO:2022-04-05 23:15:01:Please make sure that your input texts are whitespace separated tokens. The script might not work correctly otherwise.
INFO:2022-04-05 23:15:02:Generating target-association test pairs...
INFO:2022-04-05 23:15:02:Extracting candidate posts...
100%|████████████████████████████████████| 12072/12072 [01:31<00:00, 132.23it/s]
INFO:2022-04-05 23:16:33:Splitting candidate posts into sentences...
100%|████████████████████████████████████████| 33/33 [00:00<00:00, 28698.33it/s]
INFO:2022-04-05 23:16:50:Calculating sentence-based cooccurrences...
12072it [00:03, 3567.36it/s]
INFO:2022-04-05 23:16:53:Unpacking cooccurrences into a more useful format...
INFO:2022-04-05 23:16:53:Exporting results to disk at output/weat_cooccurrence_analysis/weat-cooccurrence-analysis_results-20220405231653.json.
Done.


In [48]:
!python accumulate_cooccurrence_counts.py \
    --input "output/weat_cooccurrence_analysis/weat-cooccurrence-analysis_results-20220405231653.json" \
    --weat_lexicons "sbeval/tests/weat_tests_german.json"

INFO:2022-04-05 23:17:49:Printing accumulated results for corpus 'reviews_glove.txt'.


Current test: test1
---------------------------------------------
X Blumen
Association-A:Association-B
0:0
Y Insekten
Association-A:Association-B
0:0


Current test: test2
---------------------------------------------
X Instrumente
Association-A:Association-B
0:0
Y Waffen
Association-A:Association-B
0:0


Current test: test3
---------------------------------------------
X Europäisch-amerikanische
Association-A:Association-B
3:0
Y afro-amerikanische Namen
Association-A:Association-B
0:0


Current test: test4
---------------------------------------------
X Europäisch-amerikanische
Association-A:Association-B
0:0
Y afro-amerikanische Namen
Association-A:Association-B
0:0


Current test: test5
---------------------------------------------
X Europäisch-amerikanische
Association-A:Association-B
1:0
Y afro-amerikanische Namen
Association-A:Association-B
0:0


Current test: test6
---------------------------

In [46]:
!python weat_cooccurrence_analysis_german.py \
    --data "essays_glove.txt" \
    --output "output/weat_cooccurrence_analysis" \
    --processing_cores 7 \
    --tests 1 2 3 4 5 6 7 8 9 10

INFO:2022-04-05 23:16:54:Please make sure that your input texts are whitespace separated tokens. The script might not work correctly otherwise.
INFO:2022-04-05 23:16:55:Generating target-association test pairs...
INFO:2022-04-05 23:16:55:Extracting candidate posts...
100%|████████████████████████████████████| 12072/12072 [00:35<00:00, 336.39it/s]
INFO:2022-04-05 23:17:31:Splitting candidate posts into sentences...
100%|███████████████████████████████████████████| 2/2 [00:00<00:00, 1976.11it/s]
INFO:2022-04-05 23:17:47:Calculating sentence-based cooccurrences...
12072it [00:01, 7356.80it/s]
INFO:2022-04-05 23:17:49:Unpacking cooccurrences into a more useful format...
INFO:2022-04-05 23:17:49:Exporting results to disk at output/weat_cooccurrence_analysis/weat-cooccurrence-analysis_results-20220405231749.json.
Done.


In [49]:
!python accumulate_cooccurrence_counts.py \
    --input "output/weat_cooccurrence_analysis/weat-cooccurrence-analysis_results-20220405231749.json" \
    --weat_lexicons "sbeval/tests/weat_tests_german.json"

INFO:2022-04-05 23:18:04:Printing accumulated results for corpus 'essays_glove.txt'.


Current test: test1
---------------------------------------------
X Blumen
Association-A:Association-B
0:0
Y Insekten
Association-A:Association-B
0:0


Current test: test2
---------------------------------------------
X Instrumente
Association-A:Association-B
0:0
Y Waffen
Association-A:Association-B
0:0


Current test: test3
---------------------------------------------
X Europäisch-amerikanische
Association-A:Association-B
0:0
Y afro-amerikanische Namen
Association-A:Association-B
0:0


Current test: test4
---------------------------------------------
X Europäisch-amerikanische
Association-A:Association-B
0:0
Y afro-amerikanische Namen
Association-A:Association-B
0:0


Current test: test5
---------------------------------------------
X Europäisch-amerikanische
Association-A:Association-B
0:0
Y afro-amerikanische Namen
Association-A:Association-B
0:0


Current test: test6
----------------------------

In [55]:
!bash convert_glove_to_word2vec_format.sh


\e================================================================================\e
\eConverting ./GloVe/glove_models_trained/eng_essay_vectors.txt...\e

\e================================================================================\e
\eConverting ./GloVe/glove_models_trained/essay_vectors.txt...\e

\e================================================================================\e
\eConverting ./GloVe/glove_models_trained/reviews_vectors.txt...\e


In [56]:
!python embedding_bias_evaluation.py \
    --embedding_model "GloVe/glove_models_trained/eng_essay_vectors.txt" \
    --output "output/embedding_model_evaluation" \
    --lowercase

INFO:2022-04-06 10:47:29:Loading embedding model from disk.
INFO:2022-04-06 10:47:29:loading projection weights from GloVe/glove_models_trained/eng_essay_vectors.txt
INFO:2022-04-06 10:47:29:KeyedVectors lifecycle event {'msg': 'loaded (2800, 300) matrix of type float32 from GloVe/glove_models_trained/eng_essay_vectors.txt', 'binary': False, 'encoding': 'utf8', 'datetime': '2022-04-06T10:47:29.784482', 'gensim': '4.1.2', 'python': '3.9.7 | packaged by conda-forge | (default, Sep 29 2021, 19:24:02) \n[Clang 11.1.0 ]', 'platform': 'macOS-11.6-arm64-arm-64bit', 'event': 'load_word2vec_format'}
INFO:2022-04-06 10:47:29:Evaluating WEAT tests.
INFO:2022-04-06 10:47:29:Exporting results to disk at output/embedding_model_evaluation/embedding_bias_evaluation_results-20220406104729.json.
Done.


In [60]:
!python embedding_bias_evaluation_german.py \
    --embedding_model "GloVe/glove_models_trained/essay_vectors.txt" \
    --output "output/embedding_model_evaluation" \
    --lowercase

INFO:2022-04-06 10:52:34:Loading embedding model from disk.
INFO:2022-04-06 10:52:34:loading projection weights from GloVe/glove_models_trained/essay_vectors.txt
INFO:2022-04-06 10:52:35:KeyedVectors lifecycle event {'msg': 'loaded (4435, 300) matrix of type float32 from GloVe/glove_models_trained/essay_vectors.txt', 'binary': False, 'encoding': 'utf8', 'datetime': '2022-04-06T10:52:35.254396', 'gensim': '4.1.2', 'python': '3.9.7 | packaged by conda-forge | (default, Sep 29 2021, 19:24:02) \n[Clang 11.1.0 ]', 'platform': 'macOS-11.6-arm64-arm-64bit', 'event': 'load_word2vec_format'}
INFO:2022-04-06 10:52:35:Evaluating WEAT tests.
INFO:2022-04-06 10:52:35:Exporting results to disk at output/embedding_model_evaluation/embedding_bias_evaluation_results-20220406105235.json.
Done.


In [61]:
!python embedding_bias_evaluation_german.py \
    --embedding_model "GloVe/glove_models_trained/reviews_vectors.txt" \
    --output "output/embedding_model_evaluation" \
    --lowercase

INFO:2022-04-06 10:52:39:Loading embedding model from disk.
INFO:2022-04-06 10:52:39:loading projection weights from GloVe/glove_models_trained/reviews_vectors.txt
INFO:2022-04-06 10:52:40:KeyedVectors lifecycle event {'msg': 'loaded (12592, 300) matrix of type float32 from GloVe/glove_models_trained/reviews_vectors.txt', 'binary': False, 'encoding': 'utf8', 'datetime': '2022-04-06T10:52:40.951094', 'gensim': '4.1.2', 'python': '3.9.7 | packaged by conda-forge | (default, Sep 29 2021, 19:24:02) \n[Clang 11.1.0 ]', 'platform': 'macOS-11.6-arm64-arm-64bit', 'event': 'load_word2vec_format'}
INFO:2022-04-06 10:52:40:Evaluating WEAT tests.
INFO:2022-04-06 10:52:40:Exporting results to disk at output/embedding_model_evaluation/embedding_bias_evaluation_results-20220406105240.json.
Done.
